In [1]:
import pandas as pd
import pandas as pd

df_cleaned = pd.read_csv("data/processed/labeled_time_series.csv")
df_cleaned["date"] = pd.to_datetime(df_cleaned["date"])
df_cleaned.head()

# 0) date format
df_cleaned["date"] = pd.to_datetime(df_cleaned["date"])

# 1) feature cols
META = {"Country", "date"}
feature_cols = [c for c in df_cleaned.columns if c not in META]

# 2) rolling cutoffs
max_date = df_cleaned["date"].max()
cutoff_3y = max_date - pd.DateOffset(years=3)
cutoff_2y = max_date - pd.DateOffset(years=2)
cutoff_6m = max_date - pd.DateOffset(months=6)

df_3y_raw = df_cleaned[df_cleaned["date"] >= cutoff_3y].copy()
df_2y_raw = df_cleaned[df_cleaned["date"] >= cutoff_2y].copy()
df_6m_raw = df_cleaned[df_cleaned["date"] >= cutoff_6m].copy()

# 3) coverage check (%60+ week coverage)
def check_coverage(df_raw, name, threshold=0.6):
    total_weeks = df_raw["date"].nunique()
    cov = df_raw.groupby("Country")["date"].nunique() / total_weeks
    valid = cov[cov >= threshold].index
    print(f"{name} - %{int(threshold*100)}+ coverage: {len(valid)}/{df_raw['Country'].nunique()} (weeks={total_weeks})")
    return valid

valid_3y = check_coverage(df_3y_raw, "3y")
valid_2y = check_coverage(df_2y_raw, "2y")
valid_6m = check_coverage(df_6m_raw, "6m")

# 4) aggregate to 1 row per country (median recommended for trends)
AGG = "median"   # "mean" is also ok

def agg_country(df_raw, valid, agg="median"):
    df_raw = df_raw[df_raw["Country"].isin(valid)]
    if agg == "median":
        return df_raw.groupby("Country")[feature_cols].median().reset_index()
    elif agg == "mean":
        return df_raw.groupby("Country")[feature_cols].mean().reset_index()
    else:
        raise ValueError("agg must be 'median' or 'mean'")

df_3y = agg_country(df_3y_raw, valid_3y, agg=AGG)
df_2y = agg_country(df_2y_raw, valid_2y, agg=AGG)
df_6m = agg_country(df_6m_raw, valid_6m, agg=AGG)

print("Shapes:")
print("df_3y:", df_3y.shape, "| df_2y:", df_2y.shape, "| df_6m:", df_6m.shape)

datasets = {"3y": df_3y, "2y": df_2y, "6m": df_6m}


3y - %60+ coverage: 70/70 (weeks=157)
2y - %60+ coverage: 70/70 (weeks=105)
6m - %60+ coverage: 70/70 (weeks=27)
Shapes:
df_3y: (70, 179) | df_2y: (70, 179) | df_6m: (70, 179)


In [2]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import umap
import hdbscan

OUT_DIR = "data/processed/clustering_umap_hdbscan"
os.makedirs(OUT_DIR, exist_ok=True)

final_results_hdb = {}
final_results_km  = {}
reports = {}

for name, data in datasets.items():
    print(f"\n=== {name.upper()} WINDOW ===")

    # --- 1) features (Country 제외)
    assert "Country" in data.columns
    feat_cols = [c for c in data.columns if c != "Country"]

    # --- 2) numeric + missing
    X_raw = data[feat_cols].apply(pd.to_numeric, errors="coerce")
    X = X_raw.fillna(X_raw.median(numeric_only=True)).values

    # --- 3) scaling
    X_scaled = RobustScaler().fit_transform(X)

    # --- 4) UMAP (safe)
    n_samples, n_features = X_scaled.shape
    n_neighbors  = min(15, max(2, n_samples - 1))
    n_components = min(10, n_features)  # 24 feature varsa 10 gayet yeterli

    reducer = umap.UMAP(
        n_neighbors=n_neighbors,
        n_components=n_components,
        min_dist=0.1,
        random_state=42,
        metric="euclidean"
    )
    X_umap = reducer.fit_transform(X_scaled)
    print(f"UMAP: {X_umap.shape} (neighbors={n_neighbors}, components={n_components})")

    # --- 5) HDBSCAN
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=5,
        min_samples=3,
        cluster_selection_method="eom"
    )
    labels_hdb = clusterer.fit_predict(X_umap)

    n_clusters_hdb = len(set(labels_hdb)) - (1 if -1 in labels_hdb else 0)
    n_noise = int(np.sum(labels_hdb == -1))
    print(f"HDBSCAN: {n_clusters_hdb} clusters, noise={n_noise}")

    # silhouette for HDBSCAN (ignore noise, only if >=2 clusters)
    hdb_sil = None
    mask = labels_hdb != -1
    if n_clusters_hdb >= 2 and mask.sum() >= 5:
        hdb_sil = float(silhouette_score(X_umap[mask], labels_hdb[mask]))
        print(f"HDBSCAN silhouette (non-noise): {hdb_sil:.3f}")

    out_hdb = data[["Country"]].copy()
    out_hdb["cluster_hdbscan"] = labels_hdb
    final_results_hdb[name] = out_hdb

    # --- 6) Optimal KMeans on UMAP
    # k must be <= n_samples-1
    k_min = 2
    k_max = min(15, n_samples - 1)

    km_best_k = None
    km_best_sil = None
    labels_km_best = None

    if k_max >= k_min:
        K = list(range(k_min, k_max + 1))
        sils = []
        for k in K:
            km = KMeans(n_clusters=k, random_state=42, n_init=25)
            labels = km.fit_predict(X_umap)
            sils.append(float(silhouette_score(X_umap, labels)))

        best_idx = int(np.argmax(sils))
        km_best_k = K[best_idx]
        km_best_sil = sils[best_idx]

        km_best = KMeans(n_clusters=km_best_k, random_state=42, n_init=50)
        labels_km_best = km_best.fit_predict(X_umap)

        print(f"Optimal KMeans: k={km_best_k}, silhouette={km_best_sil:.3f}")
        print(pd.Series(labels_km_best).value_counts().sort_index())

        out_km = data[["Country"]].copy()
        out_km["cluster_kmeans_umap"] = labels_km_best
        final_results_km[name] = out_km

        # save k-search report
        pd.DataFrame({"k": K, "silhouette": sils}).sort_values("silhouette", ascending=False) \
            .to_csv(os.path.join(OUT_DIR, f"kmeans_silhouette_{name}.csv"), index=False)
    else:
        print("KMeans skipped: not enough countries for k>=2.")
        out_km = data[["Country"]].copy()
        out_km["cluster_kmeans_umap"] = 0
        final_results_km[name] = out_km

    # --- 7) save outputs per window
    final_results_hdb[name].to_csv(os.path.join(OUT_DIR, f"countries_hdbscan_{name}.csv"), index=False)
    final_results_km[name].to_csv(os.path.join(OUT_DIR, f"countries_kmeans_umap_{name}.csv"), index=False)

    reports[name] = {
        "n_countries": int(n_samples),
        "n_features": int(n_features),
        "umap_neighbors": int(n_neighbors),
        "umap_components": int(n_components),
        "hdbscan_clusters": int(n_clusters_hdb),
        "hdbscan_noise": int(n_noise),
        "hdbscan_silhouette_non_noise": hdb_sil,
        "kmeans_best_k": km_best_k,
        "kmeans_best_silhouette": km_best_sil,
    }

pd.DataFrame.from_dict(reports, orient="index").to_csv(os.path.join(OUT_DIR, "run_report.csv"), index=True)
print("\nBitti. Çıktılar:", OUT_DIR)



=== 3Y WINDOW ===


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP: (70, 10) (neighbors=15, components=10)
HDBSCAN: 3 clusters, noise=17
HDBSCAN silhouette (non-noise): 0.411
Optimal KMeans: k=2, silhouette=0.346
0    34
1    36
Name: count, dtype: int64

=== 2Y WINDOW ===


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP: (70, 10) (neighbors=15, components=10)
HDBSCAN: 2 clusters, noise=17
HDBSCAN silhouette (non-noise): 0.173
Optimal KMeans: k=2, silhouette=0.290
0    21
1    49
Name: count, dtype: int64

=== 6M WINDOW ===


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP: (70, 10) (neighbors=15, components=10)
HDBSCAN: 3 clusters, noise=24
HDBSCAN silhouette (non-noise): 0.364
Optimal KMeans: k=2, silhouette=0.313
0    39
1    31
Name: count, dtype: int64

Bitti. Çıktılar: data/processed/clustering_umap_hdbscan
